<a href="https://colab.research.google.com/github/arthurst38/deep_learning/blob/main/R%C3%A9seau_de_neurones_avec_TensorFlow_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Réseau de neurones avec TensorFlow 2

Dans ce TP, nous allons construire plusieurs réseaux de neurones : un simple d'abord (sans couche cachée), puis un perceptron multicouches (plusieurs couches cachées). Nous utiliserons le dataset MNIST pour tester l'apprentissage de ce réseau en conditions réelles.

MNIST est le dataset « Hello world » du machine learning. Il est composé d'images de 28 $\times$ 28 pixels en niveaux de gris. Ces images représentent des chiffres (0 à 9). Chaque image est associée à un label indiquant le caractère que l'image est sensée représenter.

## Imports des librairies

Tensorflow et le dataset MNIST


In [ ]:
import functools
import itertools
import random
import typing

import numpy
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import tensorflow as tf
import tensorflow.keras as keras

`matplotlib`, `numpy` et `seaborn` sont des librairies de base en machine learning avec Python que nous utiliserons ici pour la visualisation et les opérations simples sur des matrices

## Récupération des données

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

## Regardons les données

Utilisez la cellule suivante pour manipuler l'objet 'mnist'. N'hésitez pas à utiliser la complétion automatique pour parcourir ses différents attributs et méthodes.

Afin de faire du machine learning, on cherche des ensembles de train et de test et à afficher quelques exemples avec leurs labels associés.

Vous essayerez de répondre aux questions suivantes :
- Combien y a-t-il d'images au total dans ce dataset ?
- Sous quelle forme sont stockées les images ? les labels ?
- Les classes sont-elles équilibrées ?

In [ ]:
# Votre code ici

### Solution

In [ ]:
example = X_train[0]
example_label = y_train[0]

print(f"Format des exemples : {X_train.shape}")
print(f"Format des labels : {y_train.shape}")

plt.imshow(example, cmap="gray_r")
plt.title(f"Premier exemple du dataset ({example_label})")
plt.show()

seaborn.countplot(x=y_train)
plt.title("Décompte des différentes classes (chiffres)")
plt.ylabel("Décompte")
plt.xlabel("Chiffre")
plt.show()

## Affichage d'exemples

Affichez 25 exemples, tirés au hasard dans la base de train. Vous n'oublierez pas d'afficher le label correspondant sous une forme facile à lire.

Pour cela utilisez :
- [`numpy.random.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)
- [`matplotlib.pyplot.imshow`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html)
- [`matplotlib.pyplot.subplots`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html) pour un affichage élégant

In [ ]:
# Utilisation des subplots pour un affichage en grande grille 5x5
f, ax = plt.subplots(5, 5, figsize=(15, 15))

# Votre code ici

# ax[2, 4].imshow(... , cmap="gray_r")
# ax[2, 4].set_title("Exemple n (label)")

### Solution

In [ ]:
# Création de la grille de sous-plots. On donne l'argument figsize pour agrandir
# la taille de la figure qui est petite par défaut
f, ax = plt.subplots(5, 5, figsize=(15, 15))

# On choisit 25 indices au hasard, sans replacement (on ne veut pas afficher la
# même image deux fois)
random_indexes = numpy.random.choice(X_train.shape[0],
                                     size=(5, 5),
                                     replace=False)

for i in range(5):
  for j in range(5):
    img_index = random_indexes[i, j]
    image = X_train[img_index]
    label = y_train[img_index]

    # Affichage avec matplotlib et sa fonction imshow, très pratique en vision par
    # ordinateur
    ax[i, j].imshow(image, cmap='gray_r')
    ax[i, j].set_title(f"Exemple {img_index} ({label})")
    ax[i, j].axis('off')

## Travail des données

Les données telles quelles sont peu manipulables : pour rendre leur utilisation plus facile dans un réseau de neurones, on peut les faire passer de leurs dimensions originales `(batch, 28, 28)` à `(batch, 28²)`. On peut aussi remarquer que leur type est `uint8` : nous allons utiliser des poids pour nos réseaux de neurones qui sont incompatibles (`float32`). On peut donc d'ores et déjà convertir les type de ces données vers `float32`.

De plus, les réseaux de neurones (comme leur version la plus simple, la régression linéaire) travaillent mieux sur des valeurs proches de 0.

- *Passez la forme de `X_train` et `X_test` de (batch, 28, 28) à (batch, 28²)*
- *Convertissez les tableaux numpy de type `uint8` en tenseurs TensorFlow de type `float32`*
- *Centrez les tenseurs sur 0 et normalisez par l'écart type*

In [ ]:
# Votre code ici

### Solution

In [ ]:
# Appel à reshape et astype de numpy. -1 signifie pour reshape de « remplir » la
# dimension avec tous les éléments restants
X_train = X_train.reshape(X_train.shape[0], -1).astype("float32")
X_test = X_test.reshape(X_test.shape[0], -1).astype("float32")

# Calcul de la moyenne et de l'écart type pour pouvoir normaliser
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

# Certaines colonnes sont constantes. On pourrait les supprimer. Ici, on décide
# plutôt de les laisser à 0 (il faut en tout cas faire quelque chose sinon on
# divise par 0 en normalisant)
X_std[X_std == 0] = 1

X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = tf.constant(X_train)
X_test = tf.constant(X_test)

## Construction d'un réseau de neurones sans couche cachée

### Variables du modèle

Dans un réseau de neurones sans couche cachée, $y = \sigma(XW +b)$, où $\sigma$ est une fonction adaptée au problème. Ici, on choisira la fonction softmax, étant donné que nous nous attaquons à un problème de classification multi-classes.

*Complétez la fonction `create_weights` pour initialiser `W` et `b` comme [variables TensorFlow](https://www.tensorflow.org/guide/variable) avec la fonction [`tensorflow.random.normal`](https://www.tensorflow.org/api_docs/python/tf/random/normal).*

In [ ]:
def create_weights(n_inputs: int,
                   n_outputs: int
                  ) -> typing.Tuple[tf.Variable, tf.Variable]:
  W = None  # Votre code ici
  b = None  # Votre code ici
  return W, b

#### Solution

In [ ]:
def create_simple_nn_weights(n_inputs: int,
                             n_outputs: int
                            ) -> typing.Tuple[tf.Variable, tf.Variable]:
  return (tf.Variable(tf.random.normal((n_inputs, n_outputs)), name="W"),
          tf.Variable(tf.random.normal((n_outputs,)), name="b"))

### Définition du modèle

Nous allons ici préparer l'architecture de notre modèle.

*Corrigez le code suivant afin que `y_pred` soit le résultat de notre réseau de neurones sans couches cachées : $\text{softmax}(XW+b)$. On utilisera le [log du softmax](https://www.tensorflow.org/api_docs/python/tf/nn/log_softmax) plutôt que le softmax standard pour plus de stabilité numérique.*

In [ ]:
class SimpleNN(tf.Module):
  def __init__(self, n_inputs: int = 28 ** 2, n_outputs: int = 10):
    super().__init__()
    self.W, self.b = create_simple_nn_weights(n_inputs, n_outputs)

  def __call__(self, X: tf.Tensor):
    y_pred = X  # Votre code ici
    return y_pred


SimpleNN()(X_train)

#### Solution

In [ ]:
# Creation du modèle
class SimpleNN(tf.Module):
  def __init__(self, n_inputs: int = 28 ** 2, n_outputs: int = 10):
    super().__init__()
    self.W, self.b = create_simple_nn_weights(n_inputs, n_outputs)

  def __call__(self, X: tf.Tensor):
    return tf.nn.log_softmax(X @ self.W + self.b)


SimpleNN()(X_train)

### Calcul de la fonction de perte

Nous allons utiliser une fonction de perte standard en classification multi-classes : l'entropie croisée. Pour cela vous pouvez faire appel à [`tensorflow.keras.losses.sparse_categorical_crossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/sparse_categorical_crossentropy). De plus on souhaite pouvoir interpréter directement cette valeur, il faut donc que la fonction `categorical_crossentropy` rende une valeur aggrégée (la moyenne), ce que ne fait pas la fonction de Keras.

Vous pourrez utiliser pour cela [`tensorflow.math.reduce_mean`](https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean).

*Complétez la fonction `categorical_crossentropy`.*

In [ ]:
def categorical_crossentropy(X: tf.Tensor,
                             y: tf.Tensor,
                             model: typing.Any
                            ) -> tf.Tensor:
  forward = model(X)
  loss = None  # Votre code ici
  return loss


categorical_crossentropy(X_train, y_train, SimpleNN())

#### Solution

In [ ]:
def categorical_crossentropy(X: tf.Tensor,
                             y: tf.Tensor,
                             model: typing.Any
                            ) -> tf.Tensor:
  forward = model(X)
  cross_entropy = keras.losses.sparse_categorical_crossentropy(y,
                                                               forward,
                                                               from_logits=True)
  return tf.reduce_mean(cross_entropy)


categorical_crossentropy(X_train, y_train, SimpleNN())

### Métriques

Pour savoir si un modèle apprend correctement, il est important de mesurer ses performances. Dans ces travaux pratiques, nous allons utiliser la performance la plus simple mais aussi une des plus informatives : l'accuracy. C'est simplement le nombre de bonnes prédictions sur le nombre total de prédictions.

*Utilisez [numpy.argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) et les comparaisons de tableaux numpy pour compléter la fonction accuracy.*

In [ ]:
def accuracy(y: tf.Tensor, y_pred: tf.Tensor) -> float:
  predictions = None  # Votre code ici
  return None


accuracy(y_test, SimpleNN()(X_test))

#### Solution

In [ ]:
def accuracy(y: tf.Tensor, y_pred: tf.Tensor) -> float:
  predictions = y_pred.numpy().argmax(axis=-1)
  same_predictions = (predictions == y).sum()
  return same_predictions / y.shape[0]


accuracy(y_test, SimpleNN()(X_test))

## Création d'un dataset TensorFlow

- *Utilisez [`tensorflow.data.Dataset.from_tensor_slices`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) pour coder la fonction `create_dataset` qui crée un dataset TensorFlow à partir des tableaux NumPy `X_train` et `y_train`. Chaque exemple doit être un dictionnaire qui contient les clefs `x` et `y`.*
- *Utilisez la fonction [`batch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch)* pour regrouper les exemples en batchs à l'intérieur du dataset.*
- *Utilisez la fonction [`shuffle`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) pour que le dataset soit mélangé à chaque itération.*

In [ ]:
def create_dataset(X: tf.Tensor, y: tf.Tensor, batch_size: int = 4000
                  ) -> tf.data.Dataset:
  dataset = None  # Votre code ici
  return dataset

### Solution

In [ ]:
def create_dataset(X: tf.Tensor, y: tf.Tensor, batch_size: int = 4000
                  ) -> tf.data.Dataset:
  dataset = tf.data.Dataset.from_tensor_slices({"x": X_train,
                                                "y": y_train})
  dataset = dataset.batch(batch_size)
  dataset = dataset.shuffle(10_000, reshuffle_each_iteration=True)
  return dataset

## Apprentissage

Voici une boucle d'apprentissage quasiment mise en place.

*Implémentez la partie manquante, qui procède à la mise à jour des paramètres pour un batch donné. Vous utiliserez pour cela [`tensorflow.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape).*

In [ ]:
def train(model: typing.Any,
          X_train: tf.Tensor = X_train,
          y_train: tf.Tensor = y_train,
          X_test: tf.Tensor = X_test,
          y_test: tf.Tensor = y_test,
          epochs: int = 150,
          batch_size: int = 4000,
          learning_rate: float = 0.001,
          evaluate_every: int = 10,
          loss_function = categorical_crossentropy
         ) -> typing.Tuple[typing.List[float], ...]:
  accuracies = [accuracy(y_train, model(X_train))]
  val_accuracies = [accuracy(y_test, model(X_test))]
  losses = [loss_function(X_train, y_train, model)]
  val_losses = [loss_function(X_test, y_test, model)]

  print(f"Métriques initiales : acc {accuracies[-1]:.4f}, "
        f"val_acc {val_accuracies[-1]:.4f}, "
        f"loss {losses[-1]:.4f}, "
        f"val_loss {val_losses[-1]:.4f}")

  dataset_train = create_dataset(X_train, y_train)

  for e in range(epochs):

    for batch in dataset_train.as_numpy_iterator():

      # Votre code ici
      pass

    # Calcul et affichage de la métrique d'évaluation sur l'ensemble de
    # validation toutes les `evaluate_every` epochs
    if (e + 1) % evaluate_every == 0:
      accuracies.append(accuracy(y_train, model(X_train)))
      losses.append(loss_function(X_train, y_train, model))
      val_accuracies.append(accuracy(y_test, model(X_test)))
      val_losses.append(loss_function(X_test, y_test, model))
      print(f"Métriques epoch {e + 1} : acc {accuracies[-1]:.4f}, "
            f"val_acc {val_accuracies[-1]:.4f}, "
            f"loss {losses[-1]:.4f}, "
            f"val_loss {val_losses[-1]:.4f}")

  x_ticks = numpy.arange(0, epochs + 1, evaluate_every)

  plt.plot(x_ticks, losses, label="Train")
  plt.plot(x_ticks, val_losses, label="Val")
  plt.title("Fonction de perte pendant l'entrainement")
  plt.xlabel("Epochs")
  plt.legend(loc="upper right")
  plt.show()
  plt.plot(x_ticks, accuracies, label="Train")
  plt.plot(x_ticks, val_accuracies, label="Val")
  plt.title("Accuracy pendant l'entrainement")
  plt.xlabel("Epochs")
  plt.legend(loc="upper right")
  plt.show()
  return accuracies, val_accuracies, losses, val_losses


_ = train(SimpleNN(), epochs=100, learning_rate=3)

### Solution

In [ ]:
def train(model: typing.Any,
          X_train: tf.Tensor = X_train,
          y_train: tf.Tensor = y_train,
          X_test: tf.Tensor = X_test,
          y_test: tf.Tensor = y_test,
          epochs: int = 150,
          batch_size: int = 4000,
          learning_rate: float = 0.001,
          evaluate_every: int = 10,
          loss_function = categorical_crossentropy
         ) -> typing.Tuple[typing.List[float], ...]:
  accuracies = [accuracy(y_train, model(X_train))]
  val_accuracies = [accuracy(y_test, model(X_test))]
  losses = [loss_function(X_train, y_train, model)]
  val_losses = [loss_function(X_test, y_test, model)]

  print(f"Métriques initiales : acc {accuracies[-1]:.4f}, "
        f"val_acc {val_accuracies[-1]:.4f}, "
        f"loss {losses[-1]:.4f}, "
        f"val_loss {val_losses[-1]:.4f}")

  dataset_train = create_dataset(X_train, y_train)

  for e in range(epochs):

    for batch in dataset_train.as_numpy_iterator():

      # Calcul de la perte avec enregistrement des opérations
      with tf.GradientTape() as tape:
        loss = loss_function(batch["x"], batch["y"], model)

      # Utilisation de l'enregistrement pour calculer automatiquement le gradient
      gradients = tape.gradient(loss, model.variables)

      # Parcours des variables une par une pour les mettre à jour en suivant la
      # règle : nouvelle_valeur = ancienne_valeur - learning_rate * gradient
      for gradient, variable in zip(gradients, model.trainable_variables):
        variable.assign_sub(gradient * learning_rate)

    # Calcul et affichage de la métrique d'évaluation sur l'ensemble de
    # validation toutes les `evaluate_every` epochs
    if (e + 1) % evaluate_every == 0:
      accuracies.append(accuracy(y_train, model(X_train)))
      losses.append(loss_function(X_train, y_train, model))
      val_accuracies.append(accuracy(y_test, model(X_test)))
      val_losses.append(loss_function(X_test, y_test, model))
      print(f"Métriques epoch {e + 1} : acc {accuracies[-1]:.4f}, "
            f"val_acc {val_accuracies[-1]:.4f}, "
            f"loss {losses[-1]:.4f}, "
            f"val_loss {val_losses[-1]:.4f}")

  x_ticks = numpy.arange(0, epochs + 1, evaluate_every)

  plt.plot(x_ticks, losses, label="Train")
  plt.plot(x_ticks, val_losses, label="Val")
  plt.title("Fonction de perte pendant l'entrainement")
  plt.xlabel("Epochs")
  plt.legend(loc="upper right")
  plt.show()
  plt.plot(x_ticks, accuracies, label="Train")
  plt.plot(x_ticks, val_accuracies, label="Val")
  plt.title("Accuracy pendant l'entrainement")
  plt.xlabel("Epochs")
  plt.legend(loc="upper right")
  plt.show()
  return accuracies, val_accuracies, losses, val_losses


_ = train(SimpleNN(), epochs=100, learning_rate=3)

## Passage à des réseaux profonds

Pour passer à des réseaux profonds, on ajoute des matrices de poids et de biais pour chaque couche à notre fonction de création de poids, par exemple comme suit :

In [ ]:
def create_mlp_weights(n_inputs: int,
                       n_outputs: int,
                       hidden_layer_sizes: typing.List[int] = [512],
                      ) -> typing.List[typing.Dict[str, tf.Variable]]:
  variables = []

  # On garde en mémoire la dernière taille de sortie : ça sera la nouvelle
  # taille d'entrée. Vaut n_inputs au départ
  last_size = n_inputs

  for i, hidden_layer_size in enumerate(hidden_layer_sizes, 1):
    # On initialise aléatoirement une matrice de poids et un vecteur de biais
    W = tf.random.normal((last_size, hidden_layer_size))
    b = tf.zeros((hidden_layer_size,))

    # On ajoute ces deux variables dans un dictionnaire puis dans la liste de
    # nos variables
    variables.append(dict(W=tf.Variable(W), b=tf.Variable(b)))

    # On met à jour la dernière taille de sortie utilisée
    last_size = hidden_layer_size

  # La dernière couche est spéciale : elle fait toujours n_outputs en taille de
  # sortie, on la traite donc à part
  W = tf.random.normal((last_size, n_outputs))
  b = tf.zeros((n_outputs,))
  variables.append(dict(W=tf.Variable(W), b=tf.Variable(b)))
  return variables

Pour ce qui est de la sortie du modèle, elle est maintenant calculée itérativement, couche par couche. Chaque couche prend en entrée le résultat de la couche précédente. Par simplicité, vous pourrez fixer vous même la fonction d'activation que vous utiliserez pour les couches cachées.

*Complétez la fonction `MLP.__call__` pour calculer le résultat d'un réseau profond. Chaque variable est accessible grâce à `self.weights[layer_number][variable_name]`. Par exemple, pour accéder à la matrice `W` du premier layer : `self.weights[0]["W"]`.*

In [ ]:
class MLP(tf.Module):
  def __init__(self,
               hidden_layer_sizes: typing.List[int] = [512],
               n_inputs: int = 28 ** 2,
               n_outputs: int = 10):
    super().__init__()
    self.weights = create_mlp_weights(n_inputs, n_outputs, hidden_layer_sizes)

  def __call__(self, X: tf.Tensor):
    # Votre code ici
    return X

### Solution

In [ ]:
class MLP(tf.Module):
  def __init__(self,
               hidden_layer_sizes: typing.List[int] = [512],
               n_inputs: int = 28 ** 2,
               n_outputs: int = 10):
    super().__init__()
    self.weights = create_mlp_weights(n_inputs, n_outputs, hidden_layer_sizes)

  def __call__(self, X: tf.Tensor):
    current = X
    for layer_weights in self.weights:
      # Calcul de la sortie de la couche de neurones non activée
      current = current @ layer_weights["W"] + layer_weights["b"]
      # Activation si on n'est pas dans la couche de sortie
      if layer_weights is not self.weights[-1]:
        current = tf.nn.tanh(current)
    return tf.nn.log_softmax(current)

## Entraînement du réseau profond

In [ ]:
_ = train(MLP(), learning_rate=0.5, epochs=100)

*Que constatez-vous en entraînant un réseau avec une couche cachée de 512 neurones ?*

Votre réponse ici

### Solution

Le réseau apprend moins bien que le réseau précédent qui n'avait pas de couche cachée : il a une loss quasi-parfaite sur l'ensemble d'entraînement mais commet quasiment 50% d'erreurs en plus sur la validation. C'est un cas de sur-apprentissage.

## Régularisation

La régularisation est un bon moyen de lutter contre le phénomène que l'on constate en entrainant notre réseau avec une couche cachée.

Pour la mettre en place, on rajoute un terme à la fonction de perte qui pénalise les valeurs importantes des poids.

*Modifiez la fonction ci-dessous pour calculer la pénalité L2. Elle est égale à la somme des carrés des paramètres.*

In [ ]:
def regularized_categorical_crossentropy(X: tf.Tensor,
                                         y: tf.Tensor,
                                         model: typing.Any,
                                         weights_norm_lambda: float = 0.003
                                        ) -> tf.Tensor:
  forward = model(X)
  cross_entropy = keras.losses.sparse_categorical_crossentropy(y,
                                                               forward,
                                                               from_logits=True)
  weights_norm = tf.constant(0, dtype="float32")
  # Votre code ici
  return tf.reduce_mean(cross_entropy) + weights_norm_lambda * weights_norm


train(MLP(),
      epochs=100,
      learning_rate=0.3,
      loss_function=regularized_categorical_crossentropy)

### Solution

In [ ]:
def regularized_categorical_crossentropy(X: tf.Tensor,
                                         y: tf.Tensor,
                                         model: typing.Any,
                                         weights_norm_lambda: float = 0.003
                                        ) -> tf.Tensor:
  forward = model(X)
  cross_entropy = keras.losses.sparse_categorical_crossentropy(y,
                                                               forward,
                                                               from_logits=True)
  weights_norm = tf.constant(0, dtype="float32")
  for variable in model.variables:
    weights_norm += tf.reduce_sum(tf.math.square(variable))
  return tf.reduce_mean(cross_entropy) + weights_norm_lambda * weights_norm


train(MLP(),
      epochs=100,
      learning_rate=0.3,
      loss_function=regularized_categorical_crossentropy)

## Recherche d'hyper-paramètres

Nous allons maintenant procéder à la recherche d'hyper-paramètres. Utilisez `random.sample` pour échantillonner le learning rate et le nombre d'unités cachées et renvoyez une liste des paramètres et d'une métrique au choix.

In [ ]:
def search_hyperparameters(X_train: tf.Tensor = X_train,
                           y_train: tf.Tensor = y_train,
                           X_test: tf.Tensor = X_test,
                           y_test: tf.Tensor = y_test,
                           learning_rates: typing.List[int] = [0.1, 0.5, 1],
                           ns_hidden_units: typing.List[int]  = range(32, 513, 32),
                           n_iters: int = 5
                          ):
  params = []
  val_accs = []
  for _ in range(n_iters):
    pass
  return params, val_accs


search_hyperparameters()

### Solution

In [ ]:
def search_hyperparameters(X_train: tf.Tensor = X_train,
                           y_train: tf.Tensor = y_train,
                           X_test: tf.Tensor = X_test,
                           y_test: tf.Tensor = y_test,
                           learning_rates: typing.List[int] = [0.1, 0.5, 1],
                           ns_hidden_units: typing.List[int]  = range(32, 513, 32),
                           n_iters: int = 5
                          ):
  params = []
  val_accs = []
  for _ in range(n_iters):
    learning_rate = random.sample(learning_rates, 1)[0]
    n_hidden_units = random.sample(ns_hidden_units, 1)[0]
    mlp = MLP(hidden_layer_sizes=[n_hidden_units])
    _, val_acc_epochs, _, _ = train(mlp, X_train, y_train, X_test, y_test,
                                    learning_rate=learning_rate, epochs=50)
    val_accs.append(val_acc_epochs[-1])
    params.append(dict(learning_rate=learning_rate,
                       n_hidden_units=n_hidden_units))
  return params, val_accs


search_hyperparameters()